In [1]:
pip install python-pptx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 7.0 MB/s eta 0:00:00


In [ ]:
from pptx import Presentation
from pptx.dml.color import RGBColor
from pptx.util import Inches
from pptx.enum.shapes import MSO_SHAPE_TYPE

# Specify the path to your PowerPoint file
ppt_file = '/content/drive/MyDrive/pptfilestest/ex16.pptx'

# Create a new Presentation object
prs = Presentation(ppt_file)

# Define colors
black_color = RGBColor(0x00, 0x00, 0x00)  # Black color
white_color = RGBColor(0xFF, 0xFF, 0xFF)  # White color

# Function to check if a shape is a background shape (you may need to customize this)
def is_background_shape(shape):
    # Check if the shape is an image or shape that covers the entire slide
    if shape.shape_type in [13, 19]:  # 13: Placeholder, 19: Picture
        return True
    if shape.width >= prs.slide_width and shape.height >= prs.slide_height:
        return True
    return False

# Function to set text color to black
def set_text_color_to_black(shape):
    if shape.has_text_frame:
        for paragraph in shape.text_frame.paragraphs:
            for run in paragraph.runs:
                run.font.color.rgb = black_color

def process_shape(shape):
    # Check if the shape has a text frame and set text color to black
    if shape.has_text_frame:
        for paragraph in shape.text_frame.paragraphs:
            for run in paragraph.runs:
                run.font.color.rgb = black_color

    # If the shape is a group, recursively process each shape in the group
    if shape.shape_type == MSO_SHAPE_TYPE.GROUP:
        for sub_shape in shape.shapes:
            process_shape(sub_shape)

    # If the shape has a background, set it to white
    if hasattr(shape, 'fill') and shape.fill.type is not None:
        shape.fill.solid()
        shape.fill.fore_color.rgb = white_color

    # If the shape is a freeform (e.g., containing a formula), set the line color to black
    if shape.shape_type == MSO_SHAPE_TYPE.FREEFORM:
        print("Freeform is found")
        if shape.line:
            shape.line.color.rgb = black_color
            print("free form line color changed ")

    # Remove background shapes
    # if is_background_shape(shape):
    #     shape._element.getparent().remove(shape._element)

# Remove background shapes and set text color to black in slides
for slide in prs.slides:
    # Set text color to black for all shapes
    for shape in slide.shapes:
        process_shape(shape)
    # Remove background shapes
    # shapes_to_remove = [shape for shape in slide.shapes if is_background_shape(shape)]
    # for shape in shapes_to_remove:
    #     slide.shapes._spTree.remove(shape._element)

    # Set slide background to white
    background = slide.background
    fill = background.fill
    fill.solid()
    fill.fore_color.rgb = white_color

# Remove background from slide masters and set text color to black
for master in prs.slide_masters:
    shapes_to_remove = [shape for shape in master.shapes if is_background_shape(shape)]
    for shape in shapes_to_remove:
        master.shapes._spTree.remove(shape._element)

    # Set slide master background to white
    background = master.background
    fill = background.fill
    fill.solid()
    fill.fore_color.rgb = white_color

    #Set text color to black for all shapes
    for shape in master.shapes:
        # set_text_color_to_black(shape)
        print("shape type in master for loop. ",shape.shape_type)
        process_shape(shape)


# Remove background from slide layouts and set text color to black
for layout in prs.slide_layouts:
    shapes_to_remove = [shape for shape in layout.shapes if is_background_shape(shape)]
    for shape in shapes_to_remove:
        layout.shapes._spTree.remove(shape._element)

    # Set slide layout background to white
    background = layout.background
    fill = background.fill
    fill.solid()
    fill.fore_color.rgb = white_color

    #Set text color to black for all shapes
    for shape in layout.shapes:
        # set_text_color_to_black(shape)

        process_shape(shape)

#Save the modified presentation
output_file = '/content/drive/MyDrive/newoutput/output16_1.pptx'
prs.save(output_file)
print(f"Modified PowerPoint file saved as {output_file}")